In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
data = pd.read_csv('_Dữ_liệu_giao_dịch_ngày__202406152152.csv')

In [3]:
data.head()

,Ghép 01,Ghép 02,STT,Ngày,Mã CK,Tham chiếu,Mở cửa,Đóng cửa,Cao nhất,Thấp nhất,...,Thay đổi percent,GD khớp lệnh KL,GD khớp lệnh GT,GD thỏa thuận KL,GD thỏa thuận GT,Tổng giao dịch KL,Tổng giao dịch GT,Vốn hóa thị trường,Sàn,Ngành nghề
0,NaN,NaN,1,2022-12-30,AAA,6.86,6.89,6.83,6.90,6.78,...,-0.44,1032000,7041,0,0,1032000,7041,2610934,HOSE,NaN
1,NaN,NaN,2,2022-12-30,AAM,11.85,11.90,11.45,11.90,11.45,...,-3.38,5300,63,0,0,5300,63,119666,HOSE,NaN
2,NaN,NaN,3,2022-12-30,AAT,5.09,5.10,4.96,5.10,4.96,...,-2.55,454900,2278,0,0,454900,2278,316455,HOSE,NaN
3,NaN,NaN,4,2022-12-30,ABR,8.71,8.71,8.71,8.71,8.71,...,0.00,0,0,0,0,0,0,174200,HOSE,NaN
4,NaN,NaN,5,2022-12-30,ABS,5.35,5.47,5.22,5.47,5.20,...,-2.43,555400,2957,0,0,555400,2957,417600,HOSE,NaN


In [4]:
data.drop(['Ghép 01','Ghép 02','Ngành nghề','Thay đổi add/sub','Thay đổi percent','GD khớp lệnh GT','GD thỏa thuận KL','GD thỏa thuận GT','Tổng giao dịch KL','Tổng giao dịch GT','Vốn hóa thị trường','Sàn'], axis=1, inplace= True)

In [5]:
data.head()

,STT,Ngày,Mã CK,Tham chiếu,Mở cửa,Đóng cửa,Cao nhất,Thấp nhất,Trung bình,GD khớp lệnh KL
0,1,2022-12-30,AAA,6.86,6.89,6.83,6.90,6.78,6.82,1032000
1,2,2022-12-30,AAM,11.85,11.90,11.45,11.90,11.45,11.89,5300
2,3,2022-12-30,AAT,5.09,5.10,4.96,5.10,4.96,5.01,454900
3,4,2022-12-30,ABR,8.71,8.71,8.71,8.71,8.71,0.00,0
4,5,2022-12-30,ABS,5.35,5.47,5.22,5.47,5.20,5.32,555400


In [6]:
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất', 'Trung bình', 'GD khớp lệnh KL']


In [7]:
data=data.sort_values(by=['Mã CK', 'Ngày'])

In [8]:
data.head()

,STT,Ngày,Mã CK,Tham chiếu,Mở cửa,Đóng cửa,Cao nhất,Thấp nhất,Trung bình,GD khớp lệnh KL
0,1,2022-12-30,AAA,6.86,6.89,6.83,6.90,6.78,6.82,1032000
416,1,2023-01-03,AAA,6.83,6.81,7.15,7.15,6.81,7.04,1543300
832,1,2023-01-04,AAA,7.15,7.15,7.11,7.29,7.11,7.18,1301900
1248,1,2023-01-05,AAA,7.11,7.15,7.17,7.19,7.09,7.14,980300
1664,1,2023-01-06,AAA,7.17,7.17,7.14,7.28,7.10,7.21,1431100


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148749 entries, 0 to 148748
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   STT              148749 non-null  int64  
 1   Ngày             148749 non-null  object 
 2   Mã CK            148749 non-null  object 
 3   Tham chiếu       148749 non-null  float64
 4   Mở cửa           148749 non-null  float64
 5   Đóng cửa         148749 non-null  float64
 6   Cao nhất         148749 non-null  float64
 7   Thấp nhất        148749 non-null  float64
 8   Trung bình       148749 non-null  float64
 9   GD khớp lệnh KL  148749 non-null  int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 12.5+ MB


In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
data[features] = scaler.fit_transform(data[features])

In [11]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [12]:
def create_sequences(data, seq_length, features):
    sequences = []
    for stock in data['Mã CK'].unique():
        stock_data = data[data['Mã CK'] == stock][features].values
        for i in range(len(stock_data) - seq_length):
            sequences.append(stock_data[i:i+seq_length])
    return np.array(sequences,dtype='float64')

In [13]:
seq_length = 60
sequences = create_sequences(data, seq_length, features)
X = sequences[:, :-1, :]
y = sequences[:, -1, features.index('Đóng cửa')]

# Chia dữ liệu thành tập huấn luyện và kiểm tra
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length-1, len(features))))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1,))
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))

# Lưu mô hình
model.save('lstm_model.h5')

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 132s 42ms/step - loss: 3.6496e-04 - val_loss: 4.2927e-05
Epoch 2/3
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 130s 42ms/step - loss: 9.2802e-05 - val_loss: 2.1342e-05
Epoch 3/3
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 131s 43ms/step - loss: 8.4856e-05 - val_loss: 4.1766e-05


In [14]:
model.save('my_model.keras')